
   #### Processing Health Level 7 (HL7) Fast Healthcare Interoperability Resources (FHIR) semi-structured JSON Messages with Snowflake

   __FHIR__ was created by the standards development organization Health Level 7 (HL7) and was designed to enable health data, including clinical and administrative data, to be quickly and efficiently exchanged. 

   __FHIR R4__ is the most recent stable version of the FHIR standard, published in October 2019. It's a normative standard, meaning it's designed for forward compatibility and long-term use. FHIR R4 is widely adopted and powers many FHIR implementations today. 

   At the core of HL7 FHIR is a set of modular components called __Resources__. These form the basic data exchange format and model of FHIR. 
   Resources define the component data elements, constraints, and relationships that make up an exchangeable patient record.

Download and unzip __synthea_sample_data_fhir_r4_nov2021.zip__ file from FHIR R4 synthetic data from https://synthetichealth.github.io/synthea-sample-data/ website. The compressed archive contains synthetic HL7 FHIR R4 messages in JSON data files. 

Create the `database` and `schema` used for loading semi-structured data, and `virtual warehouse` as compute resource to load files into Snowflake and perform analytical queries.

In a separate Snowflake SQL worksheets run the SQL statements in the cell below.

In [ ]:
USE ROLE SYSADMIN;

CREATE OR REPLACE DATABASE HL7_FHIR 
    COMMENT = 'HL7 FHIR DATABASE';
    
CREATE OR REPLACE SCHEMA HL7_FHIR.HL7_FHIR_R4;

CREATE OR REPLACE WAREHOUSE HL7_FHIR_WH
    WAREHOUSE_SIZE = 'XSMALL'
    INITIALLY_SUSPENDED = FALSE
    AUTO_SUSPEND = 600
    AUTO_RESUME = TRUE
    MIN_CLUSTER_COUNT=1
    MAX_CLUSTER_COUNT=1
    SCALING_POLICY='STANDARD'
    COMMENT = '';

CREATE OR REPLACE NOTEBOOK FHIR_NB
    QUERY_WAREHOUSE=HL7_FHIR_WH
    MAIN_FILE = 'FHIR_NB.ipynb'
    WAREHOUSE=HL7_FHIR_WH;

Stage and access HL7 FHIR message. 

`Stages` are named storage locations in Snowflake used to temporarily load or unload data files before transferring them to Snowflake tables. A Snowflake stage essentially acts as a pointer or reference to a data file or set of data files. It enables Snowflake to access these data files for loading and unloading data without the need to copy or move the files.

In [ ]:
USE SCHEMA HL7_FHIR.HL7_FHIR_R4;

-- Directory tables store a catalog of staged files in cloud storage.
CREATE OR REPLACE STAGE HL7_FHIR_STAGE_INTERNAL
    DIRECTORY = (ENABLE = TRUE)
    COMMENT = 'Used for staging data';

In [ ]:
SHOW STAGES;

In [ ]:
-- DESCRIBE STAGE HL7_FHIR_STAGE_INTERNAL;

Stage data files from local file system by executing `PUT` command using `SnowSQL` client.

Invoking snowsql requires the Snowflake account identifier which can be retrieve through SnowSight UI or programatically as below.

In [ ]:
-- SELECT * FROM TABLE(FLATTEN(INPUT => PARSE_JSON(system$allowlist()))) T;

SELECT REPLACE(T.VALUE:host::VARCHAR, '.snowflakecomputing.com') AS ACCOUNT_IDENTIFIER
FROM TABLE(FLATTEN(INPUT => PARSE_JSON(system$allowlist()))) AS T
WHERE T.VALUE:type::VARCHAR = 'SNOWFLAKE_DEPLOYMENT_REGIONLESS';


In a terminal window set shell variables for account, username, role, database, schema and warehouse as per setup above, and then invoke snowsql.

`snowsql -a $SNOWSQL_ACCOUNT -u $SNOWSQL_USER -r $SNOWSQL_ROLE -d $SNOWSQL_DATABASE -s $SNOWSQL_SCHEMA -w $SNOWSQL_WAREHOUSE`

Enter user password when prompted then 

Execute PUT command to load HL7 FHIR data into stage.

`put file:///fhir/*.json @HL7_FHIR_STAGE_INTERNAL;`

557 Row(s) produced. Time Elapsed: 235.529s

Exit SnowSQL when done with CTRL+d;

In [ ]:
ALTER STAGE HL7_FHIR_STAGE_INTERNAL REFRESH;

SELECT COUNT(*) FROM DIRECTORY(@HL7_FHIR_STAGE_INTERNAL);



In [ ]:
SELECT * FROM DIRECTORY(@HL7_FHIR_STAGE_INTERNAL) LIMIT 5;


Load HL7 FHIR messages into Snowflake tables

In [ ]:
-- Create PATIENT table for HL7 data via JSON
CREATE TABLE HL7_FHIR.HL7_FHIR_R4.PATIENT
    (JSON_STRING VARIANT);

Use [JSON Editor Online](https://jsoneditoronline.org/) to view the structure of the HL7 FHIR JSON messages and facilitate creation of appropiate file format. 

A Snowflake `file format` is a named database object that encapsulates information about a data file. This information includes the file's type (CSV, JSON, etc.), formatting options, and compression method. Snowflake file formats are used to simplify the process of loading and unloading data from Snowflake tables.

In [ ]:
CREATE OR REPLACE FILE FORMAT HL7_FHIR.HL7_FHIR_R4.HL7_FHIR_JSON
    TYPE = 'JSON'
    COMPRESSION = 'AUTO'
    ENABLE_OCTAL = FALSE
    ALLOW_DUPLICATE = FALSE
    STRIP_OUTER_ARRAY = FALSE
    STRIP_NULL_VALUES = FALSE
    IGNORE_UTF8_ERRORS = FALSE;


In [ ]:
SHOW FILE FORMATS IN DATABASE HL7_FHIR;

In [ ]:
COPY INTO HL7_FHIR.HL7_FHIR_R4.PATIENT
FROM @HL7_FHIR_STAGE_INTERNAL
FILE_FORMAT = (FORMAT_NAME = 'HL7_FHIR_JSON')
ON_ERROR = 'SKIP_FILE';

In [ ]:
-- Select raw JSON
SELECT * FROM HL7_FHIR.HL7_FHIR_R4.PATIENT LIMIT 5;

Flatten JSON into a structured representation

 * `FLATTEN` is a table function that explodes compound values into multiple rows.  
 * It takes a VARIANT, OBJECT, or ARRAY column and produces a lateral view, an inline view that contains correlations to other tables that precede it in the FROM clause.
 * The returned rows consist of a fixed set of columns, that includes the `VALUE` column which contains the value of the element of flattened variant/object/array
 * FLATTEN can be used in a lateral join. When used with the `LATERAL` keyword, the inline view can contain a reference to columns in a table that precedes it in the FROM clause.

 * use `MIN` aggregate function to avoid accumulation of columns in GROUP BY clause
 * use `ANY_VALUE` aggregate function to optimize a query that has a GROUP BY clause. ANY_VALUE returns a single non-deterministic value of an expression in a group. It is optimized to return the first value.
 * `TRUNC` function truncates a DATE, TIME, or TIMESTAMP value to the specified precision.
 * `DECODE` function compares the select expression to each search expression in order. As soon as a search expression matches the selection expression, the corresponding result expression is returned.


In [ ]:
SELECT VALUE FROM PATIENT, LATERAL FLATTEN(INPUT => JSON_STRING:entry) LIMIT 1;

In [ ]:
SELECT VALUE:resource as patient_resource FROM PATIENT, LATERAL FLATTEN(INPUT => JSON_STRING:entry)
    WHERE UPPER(VALUE:resource:resourceType::STRING) = 'PATIENT' AND
    VALUE:fullUrl = 'urn:uuid:38565670-114d-6c46-50b9-76bbffb16ffb';

In [ ]:
SELECT JSON_STRING:entry[0].fullUrl::string PATIENT_ID,
 MIN(DECODE(IDENTIFIER.VALUE:type.text,'Medical Record Number',IDENTIFIER.VALUE:value::string)) MEDICAL_RECORD_NUMBER,
 UPPER(ANY_VALUE(DECODE(PATIENT_FLAT.VALUE:resource:deceasedDateTime::date,NULL,'Unknown','true'))) PATIENT_DECEASED_IND,
 TRUNC(ANY_VALUE(PATIENT_FLAT.VALUE:resource:birthDate::date),'D') PATIENT_BIRTH_DT,
 UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/StructureDefinition/patient-birthPlace',EXTENSION.VALUE:valueAddress:city::string))) PATIENT_BIRTH_CITY
 FROM PATIENT, 
    LATERAL FLATTEN(INPUT => JSON_STRING:entry) PATIENT_FLAT,
    LATERAL FLATTEN(INPUT => PATIENT_FLAT.VALUE:resource:identifier) IDENTIFIER,
    LATERAL FLATTEN(INPUT => PATIENT_FLAT.VALUE:resource:extension) EXTENSION
group by patient_id
LIMIT 1;

In [ ]:
CREATE OR REPLACE VIEW HL7_FHIR.HL7_FHIR_R4.PATIENTS_VW AS 
    SELECT
        JSON_STRING:entry[0].fullUrl::string PATIENT_ID,
        MIN(DECODE(IDENTIFIER.VALUE:type.text,'Medical Record Number',IDENTIFIER.VALUE:value::string)) PATIENT_MRN,
        MIN(DECODE(IDENTIFIER.VALUE:type.text,'Social Security Number',IDENTIFIER.VALUE:value::string)) PATIENT_SSN,
        MIN(DECODE(IDENTIFIER.VALUE:type.text,'Driver\'s License',IDENTIFIER.VALUE:value::string)) PATIENT_DRIVERS_LICENSE_NUM,
        MIN(DECODE(IDENTIFIER.VALUE:type.text,'Passport Number',IDENTIFIER.VALUE:value::string)) PATIENT_PASSPORT_NUM,
        UPPER(ANY_VALUE(UPPER(PATIENT_FLAT.VALUE:resource:name[0].family::string))) PATIENT_LAST_NM,
        UPPER(ANY_VALUE(UPPER(PATIENT_FLAT.VALUE:resource:name[0].given[0]::string))) PATIENT_FIRST_NM,
        UPPER(ANY_VALUE(UPPER(PATIENT_FLAT.VALUE:resource:name[0].prefix[0]::string))) PATIENT_NM_PREFIX,
        ANY_VALUE(UPPER(PATIENT_FLAT.VALUE:resource:gender::string)) PATIENT_SEX,
        UPPER(DECODE(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex',EXTENSION.VALUE:valueCode::string)),'M','Male',
        'F','Female',
        'Unknown')) PATIENT_BIRTH_SEX,
        UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',EXTENSION.VALUE:extension[0]:valueCoding.display::string))) PATIENT_CORE_RACE,
        UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',EXTENSION.VALUE:extension[0]:valueCoding.display::string))) PATIENT_CORE_ETHNICITY,
        UPPER(DECODE(ANY_VALUE(PATIENT_FLAT.VALUE:resource:maritalStatus.text::string),'M','Married',
        'S','Single',
        ANY_VALUE(PATIENT_FLAT.VALUE:resource:maritalStatus.text::string))) PATIENT_MARITAL_STATUS,
        TRUNC(ANY_VALUE(PATIENT_FLAT.VALUE:resource:birthDate::date),'D') PATIENT_BIRTH_DT,
        UPPER(COALESCE(ANY_VALUE(PATIENT_FLAT.VALUE:resource:multipleBirthBoolean::string),'Unknown')) PATIENT_MULTIPLE_BIRTH_IND,
        UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/StructureDefinition/patient-birthPlace',EXTENSION.VALUE:valueAddress:city::string))) PATIENT_BIRTH_CITY,
        UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/StructureDefinition/patient-birthPlace',EXTENSION.VALUE:valueAddress:country::string))) PATIENT_BIRTH_COUNTRY,
        UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/StructureDefinition/patient-birthPlace',EXTENSION.VALUE:valueAddress:state::string))) PATIENT_BIRTH_STATE,
        TRUNC(ANY_VALUE(PATIENT_FLAT.VALUE:resource:deceasedDateTime::date),'D') PATIENT_DEATH_DT,
        UPPER(ANY_VALUE(DECODE(PATIENT_FLAT.VALUE:resource:deceasedDateTime::date,NULL,'Unknown','true'))) PATIENT_DECEASED_IND,
        UPPER(MIN(DECODE(EXTENSION.VALUE:url,'http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName',EXTENSION.VALUE:valueString::string))) PATIENT_MOTHERS_MAIDEN_NAME,
        UPPER(ANY_VALUE(ADDRESS.VALUE:line[0]::string)) PATIENT_ADDR_LINE1,
        UPPER(ANY_VALUE(ADDRESS.VALUE:line[1]::string)) PATIENT_ADDR_LINE2,
        UPPER(ANY_VALUE(ADDRESS.VALUE:line[2]::string)) PATIENT_ADDR_LINE3,
        UPPER(ANY_VALUE(ADDRESS.VALUE:city::string)) PATIENT_CITY,
        UPPER(ANY_VALUE(ADDRESS.VALUE:state::string)) PATIENT_STATE,
        UPPER(ANY_VALUE(ADDRESS.VALUE:country::string)) PATIENT_COUNTRY,
        ANY_VALUE(ADDRESS.VALUE:postalCode::string) PATIENT_POSTAL_CD,                           ANY_VALUE(COALESCE(DECODE(ADDRESS.VALUE:extension[0]:extension[0].url::string,'latitude',ADDRESS.VALUE:extension[0]:extension[0].valueDecimal::float),DECODE(ADDRESS.VALUE:extension[0]:extension[1].url::string,'latitude',ADDRESS.VALUE:extension[0]:extension[1].valueDecimal::float))) PATIENT_LATITUDE,        ANY_VALUE(COALESCE(DECODE(ADDRESS.VALUE:extension[0]:extension[0].url::string,'longitude',ADDRESS.VALUE:extension[0]:extension[0].valueDecimal::float),DECODE(ADDRESS.VALUE:extension[0]:extension[1].url::string,'longitude',ADDRESS.VALUE:extension[0]:extension[1].valueDecimal::float))) PATIENT_LONGITUDE,
        MIN(DECODE(EXTENSION.VALUE:url,'http://synthetichealth.github.io/synthea/disability-adjusted-life-years',EXTENSION.VALUE:valueDecimal::string)) PATIENT_DISABILITY_ADJUSTED_LIFE_YEARS,
        MIN(DECODE(EXTENSION.VALUE:url,'http://synthetichealth.github.io/synthea/quality-adjusted-life-years',EXTENSION.VALUE:valueDecimal::string)) PATIENT_QUALITY_ADJUSTED_LIFE_YEARS,
        1 PATIENT_CNT
    FROM HL7_FHIR.HL7_FHIR_R4.PATIENT,
        LATERAL FLATTEN(INPUT => JSON_STRING:entry) PATIENT_FLAT,
        LATERAL FLATTEN(INPUT => PATIENT_FLAT.VALUE:resource:identifier) IDENTIFIER,
        LATERAL FLATTEN(INPUT => PATIENT_FLAT.VALUE:resource:extension) EXTENSION,
        LATERAL FLATTEN(INPUT => PATIENT_FLAT.VALUE:resource:address) ADDRESS
    WHERE UPPER(PATIENT_FLAT.VALUE:resource:resourceType::STRING) = 'PATIENT'
    GROUP BY PATIENT_ID;

In [ ]:
-- query the flatten views to look at HL7 FHIR JSON data in tabular format        
SELECT * FROM HL7_FHIR.HL7_FHIR_R4.PATIENTS_VW LIMIT 5;


In [ ]:
 CREATE OR REPLACE VIEW HL7_FHIR.HL7_FHIR_R4.CONDITIONS_VW AS
    SELECT
        CONDITION_FLAT.value:fullUrl::string condition_id,
        JSON_STRING:entry[0].fullUrl::string PATIENT_ID,
        CODING.VALUE:code::string CONDITION_CD,
        UPPER(CODING.VALUE:display::string) CONDITION_DESC,
        UPPER(CONDITION_FLAT.VALUE:resource.code.text::string) CONDITION_TXT,
        CONDITION_FLAT.VALUE:resource.assertedDate::date ASSERTED_DTTM,
        CONDITION_FLAT.VALUE:resource.onsetDateTime::date ONSET_DTTM,
        CONDITION_FLAT.VALUE:resource.abatementDateTime::date ABATEMENT_DTTM,
        UPPER(CONDITION_FLAT.VALUE:resource.verificationStatus::string) VERIFICATION_STATUS,
        UPPER(CONDITION_FLAT.VALUE:resource.clinicalStatus::string) CLINICAL_STATUS,
        1 CONDITION_CNT
    FROM HL7_FHIR.HL7_FHIR_R4.PATIENT, 
         LATERAL FLATTEN(INPUT => JSON_STRING:entry) CONDITION_FLAT,
         LATERAL FLATTEN(INPUT => CONDITION_FLAT.VALUE:resource.code.coding) CODING
    WHERE UPPER(CONDITION_FLAT.VALUE:request.url::string) = 'CONDITION';

In [ ]:
SELECT * FROM HL7_FHIR.HL7_FHIR_R4.CONDITIONS_VW WHERE PATIENT_ID = 'urn:uuid:76b289fd-e825-734c-8446-316f59643593';

In [ ]:
SELECT * FROM HL7_FHIR.HL7_FHIR_R4.CONDITIONS_VW WHERE CONTAINS(CONDITION_DESC, 'CANCER');

`Masking Policies` to protect Confidential Data by leveraging Dynamic Data Masking

`Dynamic Data Masking` is a Column-level Security feature that uses masking policies to selectively mask plain-text data in table and view columns at query time.

Depending on the `masking policy conditions`, the `SQL execution context`, and `role hierarchy`, Snowflake query operators may see the plain-text value, a partially masked value, or a fully masked value.

Leverage `Dynamic Data Masking` to `protect PII or HIPPA data` by `selecting different functional roles`.

`HIPAA` (Health Insurance Portability and Accountability Act) focuses on protecting Protected Health Information (PHI), while `PII` (Personally Identifiable Information) is a broader concept encompassing any data that can identify an individual, not just health-related data.

In [ ]:
CREATE OR REPLACE MASKING POLICY SIMPLE_MASK_PII_CHAR AS
    (VAL CHAR) RETURNS CHAR ->
    CASE
        WHEN CURRENT_ROLE() IN ('SYSADMIN') THEN VAL
            ELSE '***PII MASKED***'
        END;

In [ ]:
CREATE OR REPLACE MASKING POLICY SIMPLE_MASK_PII_DATE AS
    (VAL DATE) RETURNS DATE ->
    CASE
        WHEN CURRENT_ROLE() IN ('SYSADMIN') THEN VAL
            ELSE NULL
        END;

In [ ]:
CREATE OR REPLACE MASKING POLICY SIMPLE_MASK_HIPAA_CHAR AS
    (VAL CHAR) RETURNS CHAR ->
    CASE
        WHEN CURRENT_ROLE() IN ('SYSADMIN') THEN VAL
            ELSE '***HIPPA***'
        END;

In [ ]:
-- Apply masking policies to specific columns in the views
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_SSN SET MASKING POLICY SIMPLE_MASK_PII_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_DRIVERS_LICENSE_NUM SET MASKING POLICY SIMPLE_MASK_PII_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_PASSPORT_NUM SET MASKING POLICY SIMPLE_MASK_PII_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_BIRTH_DT SET MASKING POLICY SIMPLE_MASK_PII_DATE;


ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_FIRST_NM SET MASKING POLICY SIMPLE_MASK_HIPAA_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_LAST_NM SET MASKING POLICY SIMPLE_MASK_HIPAA_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_MOTHERS_MAIDEN_NAME SET MASKING POLICY SIMPLE_MASK_HIPAA_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_BIRTH_SEX SET MASKING POLICY SIMPLE_MASK_HIPAA_CHAR;
ALTER VIEW PATIENTS_VW MODIFY COLUMN PATIENT_BIRTH_CITY SET MASKING POLICY SIMPLE_MASK_HIPAA_CHAR;

In [ ]:
--Verify masking policy using SYSADMIN - can see unrestricted data
USE ROLE SYSADMIN;

SELECT PATIENT_ID, PATIENT_MRN, PATIENT_SSN, PATIENT_DRIVERS_LICENSE_NUM, PATIENT_PASSPORT_NUM, PATIENT_BIRTH_DT, 
PATIENT_BIRTH_SEX, PATIENT_BIRTH_CITY, PATIENT_FIRST_NM, PATIENT_LAST_NM, PATIENT_MOTHERS_MAIDEN_NAME 
FROM HL7_FHIR.HL7_FHIR_R4.PATIENTS_VW
LIMIT 10;

In [ ]:
--Verify masking policy using ACCOUNTADMIN - data is protected
USE ROLE ACCOUNTADMIN;

SELECT PATIENT_ID, PATIENT_MRN, PATIENT_SSN, PATIENT_DRIVERS_LICENSE_NUM, PATIENT_PASSPORT_NUM, PATIENT_BIRTH_DT, 
PATIENT_BIRTH_SEX, PATIENT_BIRTH_CITY, PATIENT_FIRST_NM, PATIENT_LAST_NM, PATIENT_MOTHERS_MAIDEN_NAME 
FROM HL7_FHIR.HL7_FHIR_R4.PATIENTS_VW
LIMIT 10;

Analytics/Visualization with Streamlit

In [ ]:
SELECT PATIENTS_VW.PATIENT_SEX, CONDITIONS_VW.CONDITION_DESC, COUNT(*) AS PATIENT_COUNT
FROM PATIENTS_VW
JOIN CONDITIONS_VW ON PATIENTS_VW.PATIENT_ID=CONDITIONS_VW.PATIENT_ID
GROUP BY PATIENT_SEX, CONDITION_DESC
ORDER BY PATIENT_COUNT DESC;


In [ ]:
import streamlit as st
import pandas as pd

pcg_data = PATIENTS_BY_CONDITION_BY_GENDER.to_pandas()  # utilizes output from the sql cell above
st.header("Patients By Condition By Gender")
st.bar_chart(pcg_data, x="CONDITION_DESC", y="PATIENT_COUNT", color="PATIENT_SEX", horizontal=False, stack=False)


In [ ]:
import plotly.express as px
import streamlit as st
import pandas as pd

pcg_data = PATIENTS_BY_CONDITION_BY_GENDER.to_pandas()  

fig = px.scatter(
    pcg_data,
    x="PATIENT_SEX",
    y="PATIENT_COUNT",
    size="PATIENT_COUNT",
    color="CONDITION_DESC",
    #hover_name="PATIENT_COUNT",
    log_y=True,
    size_max=60,
)

st.header("Patients By Condition By Gender")
st.plotly_chart(fig, theme="streamlit", use_container_width=True)


In [ ]:
SELECT PATIENT_CORE_RACE, COUNT(*) AS PATIENT_COUNT
FROM PATIENTS_VW
GROUP BY PATIENT_CORE_RACE
ORDER BY PATIENT_COUNT DESC;

In [ ]:
import streamlit as st
import pandas as pd

pcr_data = PATIENTS_BY_CORE_RACE.to_pandas() 
st.header("Patients By Condition By Gender")
st.bar_chart(pcr_data, x="PATIENT_CORE_RACE", y="PATIENT_COUNT", color="#FF991C", horizontal=True)

In [ ]:
SELECT COUNT(*) AS TOTAL_NUMBER_OF_PATIENTS FROM PATIENTS_VW;